In [105]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from fastai.text.all import *
# import PyPDF2

In [2]:
# pages_content =[]
# with open("book.pdf", "rb") as pdf_file:
#     read_pdf = PyPDF2.PdfFileReader(pdf_file)
#     for i in range(11,100):
#         page = read_pdf.pages[i] 
#         pages_content.append( page.extractText() )

In [19]:
# text = ''
# for content in pages_content:
#     text+= str(content)

In [21]:
# text =text.replace('\n','')

In [5]:
# texts = text.split(". ")

In [6]:
# for i,text in enumerate(texts):
#     texts[i] = text.strip()    

In [106]:
# [  texts.pop(i) for i,t  in enumerate(texts) if len(t) <=3]

In [107]:
path = 'book.txt'

In [108]:
file = get_text_files('.')

In [175]:
txt = file[0].open().read()

In [201]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txt)
    return ' '.join(first(sp([txt]))[:40])

In [206]:
sp = SubwordTokenizer(vocab_sz=100)
sp.setup(txt)
toks = first(sp([txt]))

In [207]:
tkn = Tokenizer(sp)

In [208]:
tkn("this is me")

['▁xxbos', '▁', 't', 'h', 'i', 's', '▁', 'i', 's', '▁', 'm', 'e']

In [125]:
toks=tkn(txt)

In [129]:
len(toks)

11430

In [133]:
num = Numericalize()
num.setup(toks)


In [134]:
num("this is me")

TensorText([10, 17, 13, 15,  0, 13, 15,  0, 18,  9])

In [135]:
nums = num(toks)

In [139]:
num("hi this is me")

TensorText([17, 13,  0, 10, 17, 13, 15,  0, 13, 15,  0, 18,  9])

In [118]:
dl = LMDataLoader([nums])

In [196]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [123]:
len(num.vocab)

56

In [172]:
' '.join(num.vocab[o] for o in x[62][:2000])

'xxunk xxunk xxunk xxunk xxunk . \n xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk . xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk . xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk . xxmaj xxunk xxunk , xxunk xxunk xxunk xxunk xxunk xxunk xxunk , xxunk xxunk xxunk . xxmaj xxunk a xxunk xxunk xxunk xxunk . \n xxmaj'

In [164]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj xxunk xxunk xxunk xxunk xxunk xxunk , “ xxunk xxunk xxunk xxunk xxunk xxunk ? ” xxmaj xxunk xxunk'

In [140]:
from datasets import Dataset,DatasetDict

In [141]:
model_nm = 'microsoft/deberta-v3-small'


In [142]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


In [144]:
tokz = AutoTokenizer.from_pretrained(model_nm)

loading configuration file config.json from cache at /home/ikram/.cache/huggingface/hub/models--microsoft--deberta-v3-small/snapshots/23bfba973812a80178eb6c2c600e85cc461ffc2c/config.json
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.22.1",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

In [145]:
def tok_func(x): return tokz(x["text"])


In [146]:
ds = Dataset.from_text(path)

Using custom data configuration default-251fae291987810a
Reusing dataset text (/home/ikram/.cache/huggingface/datasets/text/default-251fae291987810a/0.0.0)


In [147]:
tok_ds = ds.map(tok_func, batched=True)


Loading cached processed dataset at /home/ikram/.cache/huggingface/datasets/text/default-251fae291987810a/0.0.0/cache-4e23ddb602540423.arrow


In [64]:
dds = tok_ds.train_test_split(0.25, seed=42)


Loading cached split indices for dataset at /home/ikram/.cache/huggingface/datasets/text/default-251fae291987810a/0.0.0/cache-8f0eb2580e6cb948.arrow and /home/ikram/.cache/huggingface/datasets/text/default-251fae291987810a/0.0.0/cache-308fbcaf6dfa0b7e.arrow


In [161]:
dds['train'][8]

{'text': 'A hobby is something creative that’s just for you. You don’ttry to make money or get famous off it, you just do it because it makes you happy. A hobby issomething that gives but doesn’t take. ',
 'input_ids': [1,
  336,
  10169,
  269,
  491,
  1835,
  272,
  276,
  268,
  348,
  270,
  274,
  260,
  367,
  418,
  276,
  297,
  14650,
  264,
  365,
  580,
  289,
  350,
  2167,
  442,
  278,
  261,
  274,
  348,
  333,
  278,
  401,
  278,
  682,
  274,
  1005,
  260,
  336,
  10169,
  269,
  12810,
  272,
  1360,
  304,
  702,
  276,
  297,
  413,
  260,
  2],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [148]:
def corr(x,y): return np.corrcoef(x,y)[0][1]


In [149]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}


In [150]:
from transformers import TrainingArguments,Trainer


In [151]:
bs = 8
epochs = 4
lr = 8e-5

In [152]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=False,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

PyTorch: setting up devices


In [153]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

loading configuration file config.json from cache at /home/ikram/.cache/huggingface/hub/models--microsoft--deberta-v3-small/snapshots/23bfba973812a80178eb6c2c600e85cc461ffc2c/config.json
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "tra

In [162]:
trainer.train();


The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/home/ikram/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 147
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 76
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text fol

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds